# The Wilcoxon rank-sums test
### ... or the Mann-Whitney U test

*Gries, chapter 4, pp. 227-233*


#### Recap and rationale

In the previous sprints we've introduced the concept of a normal distribution, a very common model (in the shape of a bell curve), to capture the symmetric spread of a variable around a mean. We've talked about probability density functions that allow us to quantify the notorious $p$-values, describing what the probability is that a particular value was sampled from such a distribution. You understand that the further such a variable is removed from the mean of the distribution (in either direction) and the more it moves into the direction of one of the two tails of the density curve, the more unlikely it is that it was associated with the distribution.

We've also promised that this would be useful to understand how a statistical test works. However, the only thing that you know about these tests so far is that they spit out some kind of mysterious test statistic. We've asked you to trust us for the time being and not wonder how this test statistic is produced. In general, you can just assume that this number tells you something about the magnitude of the test result, e.g. it somehow captures the difference, for instance, in the length of sentences between two speakers. 

We have first covered how we can assess or evaluate that test statistic. As mentioned, conventional statistical tests are defined in very clever ways. Essentially, they have been set up in such a way that *if the null hypothesis holds*, we know **what the distribution of the test statistic should look like**. Often, this distribution is a normal and that's exactly why we had to cover the normal distribution first: this makes it easy to derive the $p$-value of the test statistic, since we can calculate whether the test statistic exceeds a certain critical threshold in that respect. If the test statistic takes a very extreme value (in the light of the distribution that we expected), we can argue that the null nypothesis should be rejected.

In this notebook, we'll talk you through a very intuitive test and present you with a very accessible example of how such a test statistic can be calculated and why we could anticipate what its behaviour should be, if the null hypothesis were true. The nice thing about this test is that it's pretty straightforward to understand but it is not some kind of simplified, naive, dummy example: it's in fact a real-world procedure that is used in a common test: this test is known as the Wilcoxon rank-sums test (or the Mann-Whitney $U$ test).

#### Case study: pitch across men with different orientations (AKA queer phonology)

In the handbook you might have seen that Gries often refers to a famous case study regarding the pitch of the voice of **two groups of men with different sexual orientation**s. The existing literature has argued that the *average* pitch between gay and straight men does not differ statistically, but there is evidence that the variance in pitch does differ significantly between the two groups, meaning that homosexual men show more variation in pitch across their speech signal than heterosexual man. We've found a dataset for Spanish speakers that allows us to look into this fascinating topic from **sociophonetics**. You can load it as follows:

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import statsmodels.api as sm
import math

In [ ]:
df = pd.read_csv("../datasets/queer/queer_speech.csv")
df

As you can, these scholars collected a whole bunch of interesting variables for each speaker. The dataset also includes (heterosexual) women, but we'll exclude these for the time being, and focus on the "M" sex:

In [ ]:
df = df[df["sex"] == "M"]
df

The column of interest here is "f0sd": this column stands, not for the average pitch (which is recorded in the column "meanf0"), but the standard deviation in pitch for this speaker. (Pitch is the auditory correlate of the so-called first formant or "F0", hence the name.) Let's limit of data frame to the pitch-related columns:

In [ ]:
df = df[["orientation", "meanf0", "f0sd"]].copy()
df

#### Visualize

Let us visualize the range of values for each category. You know how to do this from the previous session:

In [ ]:
df.plot.box(by="orientation", column=["meanf0", "f0sd"])
plt.show()

Clearly, there isn't much difference regarding the *mean* pitch per speaker category, but the *standard deviation* for the homosexual speakers seems to be much higher. Could this be true? Such a question is a vanilla application for statistical tests, since we'd like to establish that the difference between the mean pitch is (indeed?) **statistically insignificant**, just as we'd like to ascertain that the difference in standard deviation is (indeed?) **significantly different**.

Let us split the dataframe into two groups to make things easier later

In [16]:
hom_df = df[df["orientation"] == "homosexual"]
het_df = df[df["orientation"] == "heterosexual"]

The question whether the values in one group are different (viz. lower/higher) from the values in another group is very common in statistics and a number of tests would be applicable. Many of these, however, come with complex requirements of your data. Many of them will for instance critically assume that the values in both your groups are normally distributed. If you create a density plot, however, for the two groups of "f0sd", you can see that these values don't behave very "normally":

In [ ]:
# Reducing the 'bandwidth' for the density estimation fits closer to the data, increasing it gives
# us smoother curves.
sns.kdeplot(df, x="f0sd", hue="orientation", bw_adjust=0.5)

plt.show()

Like before, let's eyeball the normality using QQ plots.

In [ ]:
sm.qqplot(het_df["f0sd"], line="s")
plt.show()

In [ ]:
sm.qqplot(hom_df["f0sd"], line="s")
plt.show()

For the homosexual men, the distribution seems "kinda" Gaussian, but that's not the case for the heterosexual group. (That makes sense also, the since the variance for heterosexual group is much smaller. Think about that!) The problem is that this situation prevents us from running a whole bunch of tests that require each group to be normally distributed. Such tests, which come with many requirements about the shape of your data, are called **parametric tests**.

The Wilcoxon rank-sums test, however, is a **non-parametric test**: these tests are interesting because they come with  very few, if any, requirements and are very broadly applicable. These tests are used in very many contexts, for instance, to answer one of the classic questions in corpus linguistics, i.e. whether the frequency of a word is different in two corpora. For many reasons, the Wilcoxon rank-sums test is a test you'll learn to love, because it is: 
  - widely applicable
  - free of assumptions
  - insensitive to outliers
  - conceptually simple
  
In this next section, we'll talk you through the calculation of the test statistic -- as the name suggests, this will involve some summing of ranks. We'll start, however, by just running the test in its standard implementation in Python:

#### Running your first statistical test:

Let's recap again, because this is important. This version of Wilcoxon (or Mann-Whitney U) assess the null hypothesis that two distributions are the same. For a two-sided test, $H_0$ is "these samples come from the same distribution" and $H_1$ is "they come from different distributions". As always, a $p$-value below the critical threshold is ground to **reject $H_0$**

> In Scipy, this test is called the Mann-Whitney U test, which is the same as the Wilcoxon rank-sum test. **Be careful**, because `sp.stats.wilcoxon` does exist, but performs a different test, called the Wilcoxon signed-rank test. However, if you ever use R, the test you want *is* called `wilcoxon`. Yes, it is all very confusing.

We run the test twice: once to determine whether the difference in "meanf0" is different across the groups, and once for the difference in "f0sd". The only additional thing which we specify is that we want to carry out a "two-sided" test, indicating that our hypothesis is non-directional (i.e. we care about the effect, but not about its direction). Note that for "f0sd" we could in principle have opted for one-sided test, since we have the clear expectation that the "f0sd" would be *larger* in the heterosexual group.

In [ ]:
sp.stats.mannwhitneyu(hom_df.f0sd, het_df.f0sd, alternative="two-sided")

In [ ]:
sp.stats.mannwhitneyu(hom_df.meanf0, het_df.meanf0, alternative="two-sided")

For the one-sided test, our base group is `hom.f0sd`, $H_0$ is "the base group does not have greater variance in F0" and the alternative $H_1$ is "the base group values are stochastically *greater*".

In [ ]:
sp.stats.mannwhitneyu(hom_df.f0sd, het_df.f0sd, alternative="greater")

As with most statistical tests, two things are being returned by the test: a test statistic ($U$) and the associated $p$-value. As you already know, the test statistic captures how large the difference is (i.e. the so-called **effect size**) and the $p$-value tells us how surprised we should be by the size of that difference. Assuming that we adopt the conventional critical threshold of 0.05, we can see that:
 - in the case of the mean F0, the test gives us a p-value that is much higher than that threshold, meaning that we can *not* reject the null hypothesis in this case; 
 - the case of the variance of F0 yields a very tiny $p$-value ($p$ <<< 0.05) meaning that we should reject the null hypothesis in this case.
 
That all sounds great: but how was $U$ calculated?

#### Looking at ranks, instead of values

The test statistic that we need to calculate for this test is $W$ (or sometimes called $U$, given the alternative name Mann-Whitney $U$ of the test). As you know, the test statistic is meant to somehow capture the size of the difference between the two groups. When we would start calculating the numeric difference between the actual values in both groups, we run into the risk of hitting outliers: if one of the values in the groups would be very extreme, that would mess up the entire test statistic. Our Wilcoxon rank sums bypasses this danger by focusing on ranks, instead of the actual values.

It does this in a very specific way. We take our full data frame, select the relevant column and simply rank the values (it doesn't matter whether we do this in ascending or descending order, but the formal specification of the test states 'ascending'):

In [ ]:
df["rank_sd"] = df.f0sd.rank(ascending=True)
df["rank_mean"] = df.meanf0.rank(ascending=True)
df.sort_values(by="rank_sd")

We created two new vectors: for each of the two columns of interest, we have recorded which overall rank that value would have, if we ordered all the values in that column. Let us plot a boxplot of those ranks now (and forget about the actual values):

In [ ]:
sns.set_theme(
    context="notebook",
    style="whitegrid",
    rc={"figure.figsize": (8, 6), "figure.dpi": 144},
)

ax = sns.boxplot(df, y="rank_mean", hue="orientation")
sns.rugplot(df, y="rank_mean", hue="orientation")
sns.move_legend(ax, "lower right")

plt.show()

In [ ]:
ax = sns.boxplot(df, y="rank_sd", hue="orientation")
sns.rugplot(df, y="rank_sd", hue="orientation")
sns.move_legend(ax, "lower right")

sns.reset_defaults()
plt.show()

First of all, it's clear that the difference in medians for both categories is relatively larger in the second case (for "f0sd"). As a reading aid, we've also added small ticks (using `rugplot()`) on the vertical axis line to indicate where the ranks are located for each group; using different colors. If you look closely, you'll notice that in the first graph, there is no real difference across colors, whereas in the second plot, you'll notice that the blue stripes cluster towards the lower end.

Our test statistic $U$ is wholly calculated on the basis of the ranks of our values, instead of the actual values. By using ranks, we have the garantee that outliers won't affect our calculation much: even if a value is really high, it would just be ranked last, but that rank itself wouldn't be a very extreme value. In fact, many other non-parametric tests, such as Kendall's $\tau$ also look at ranks, instead of actual values, and this is precisely what makes them non-parametric.

#### Calculating $U$

Our test statistic $U$ is calculated by comparing the ranks in our two groups. The intuition is as follows: if there is no real difference in values across our two groups, their ranks should be **well shuffled**; in that case, we would expect that the values of one group wouldn't be ranked consistently higher than the values of the other. Let us start with the "f0sd" column: let us sort the rows of the data frame by the value of f0sd:

The "rank" column is added to make explicit which rank each rows was assigned. We can now split out these ranks across two categories:

In [26]:
ranks_hom = df.rank_sd[df.orientation == "homosexual"]
ranks_het = df.rank_sd[df.orientation == "heterosexual"]

We now want to compare the ranks in these groups: if there wouldn't be a difference across these two groups, the values for one of the groups shouldn't consistently outrank those from the other group. To calculate $W$, we therefore compare each rank from the first group ($x_i$), with every rank in the second group ($y_i$) and check whether the first rank is smaller than the second rank. $W$ then is the number of times where $x_i < y_i$. We can calculate this as follows with an explicit for-loop:

> CODING ALERT
>
> In general, if you find yourself iterating over pandas `Series` objects like this something has gone wrong and there is typically a better way. For this particular example, we absolve you of sin.

In [ ]:
U = 0
for i, hom in ranks_hom.items():
    for j, het in ranks_het.items():
        if het == hom:
            # equal rank
            U += 0.5
        elif het < hom:
            U += 1
        else:
            # do nothing
            U = U
U

You will recognize this value for $U$, since it's the test statistic being returned by the `mannwhitneyu()` above. As promised, the number gives us a sense of the overall difference between the two groups: it counts how often a rank from the first group is lower than a rank from the second group, if we exhaustively compare all ranks from the two groups.

The question now becomes how surprised we should be with this particular $U$? Is the value of 2178.5 large or pretty average? What would be the outcome that we would expect, if there were no difference whatsoever between the groups? Or put differently, even if we allow for some "wiggle" in the results, what would be the distribution of the test statistic $U$? In this case, we can guess pretty well what the expected mean $\mu$ of this distribution would be, if there were no difference:

In [ ]:
n1 = len(ranks_hom)
n2 = len(ranks_het)

mu = (n1 * n2) / 2
mu

We can calculate the number of possible pairs of ranks (each time involving one from each group). If there would be no difference in ranks, the ranks would be "shuffled" we could anticipate that in roughly half of the cases a rank from group one would be lower than a rank from group 2. That's why the mean of the expected distribution for $W$ is equal to $(n1 \cdot n2) / 2$. You see how this test is cleverly defined: we calculate a test statistic of which we know what the distribution should look like, if the null hypothesis were true.

How we can predict the expected mean $\mu$ for $W$ is intuitive, but the analogous reasoning for the standard deviation $\sigma$ is harder to understand so I'll just give you the formula for that:

In [ ]:
sigma = math.sqrt((n1 * n2) * (n1 + n2 + 1) / 12)
sigma

We now have the two crucial ingredients to plot our expected distribution and re-assess our actual test statistic in the light of this. First, let's get an intuitive feel for the distribution with bootstrap statistics:

In [ ]:
# bootstrap normal sample
rng = np.random.default_rng(seed=42)
numbers = rng.normal(loc=mu, scale=sigma, size=100000)

ax = sns.histplot(
    x=numbers,
    color="orange",
    stat="density",
    bins=50,
)
# plot the U statistics for f0sd (red) and meanf0 (blue)
ax.axvline(2178.5, color="red", linestyle="--")
ax.axvline(1288.5, color="blue", linestyle="--")

plt.show()

In [ ]:
# Some people call this a "bootstrap p-value"
1 - (sp.stats.percentileofscore(numbers, 2178.5, kind="weak") / 100)

As you can see, the test statistic value that we obtained is unlike anything we would have expected, assuming that there was no difference in ranks. This immediately explains the extremely low $p$-value that was returned by `wilcox.test()` above.


The distribution for $U$ is not a standard normal, but for large samples it is well approximated by the normal distribution. To see this, we can calculate a theoretical $p$-value like we did before, using the normal CDF for these empirical $\mu$ and $\sigma$ values:

In [ ]:
dist = sp.stats.norm(loc=mu, scale=sigma)
1 - dist.cdf(2178.5)

... and we see it is extremely close to the scipy $p$-value! Success!

In [ ]:
sp.stats.mannwhitneyu(hom_df.f0sd, het_df.f0sd, alternative="greater")

##### Exercise
> Repeat the manual calculation of the test statistic above for "meanf0" and redo the plot from the previous code block: can you reproduce the $U$ statistic that is produced by `mannwhitneyu()`?

In [ ]:
sp.stats.mannwhitneyu(hom_df.meanf0, het_df.meanf0, alternative="two-sided")

In [35]:
# code here

##### Rank sums?

This isn't crucial but you might wonder why this test is known as a "rank sums" test? How was there summing of ranks involved? That has to do with the fact that there's a smarter method to calculate $W$, instead of using an analytical for-loop, as we did above, which *does* involve summing the ranks.

In [ ]:
ranks_hom.sum() - (ranks_hom.size * (ranks_hom.size + 1) / 2)

That's all.

## Take-home message

We won't have time to cover each statistical test in the same level of detail, but we wanted to give you an intuition as to how these tests are cleverly constructed. When we calculate a given test statistic, we can formalize what its value should look like if the null hypothesis were true. Because you known in advance what the distribution of the test statistic should look like, you can calculate how surprising the actual test statistic is that you obtain. Using the combination of a $p$-value and a critical threshold (that you should set in advance, to avoid "cheating"), you can make explicit why and at which point you reject the null hypothesis $H_0$ in favour of the alternative hypothesis $H_1$. While all tests that we'll cover in the next sprint have a different underlying narrative and rationale, they all share the basic hypothesis testing principle.

<img src=https://imgs.xkcd.com/comics/null_hypothesis.png />

(from [xkcd](https://xkcd.com/892/))

```
Version History

Current: v1.0.1

23/9/24: 1.0.0: first draft, BN
04/10/24: 1.0.1: typos + proofreading, MK
```